In [126]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

import lightgbm as lgb
from lightgbm import LGBMClassifier

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from sklearn.metrics import log_loss, make_scorer
from functools import partial

from sklearn.model_selection import train_test_split

import gc
import time
import warnings
warnings.filterwarnings('ignore')

In [115]:
train = pd.read_csv('../dataset/lee_train.csv', index_col = 0)
test = pd.read_csv('../dataset/lee_test.csv', index_col = 0)
sample_submission = pd.read_csv('../dataset/sample_submission.csv', index_col = 0)

In [119]:
column_number = {}

for i, column in enumerate(sample_submission.columns):
    column_number[column] = i

def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [120]:
#bins= [0, 100, 200, 300, 400, 500, 600, 700, 800, 900 ,1000]
#labels = ['0', '1','2','3','4','5','6','7','8','9']

#train['fiberID2'] = pd.cut(train['fiberID'], bins= bins, labels = labels)
#test['fiberID2'] = pd.cut(test['fiberID'], bins= bins, labels = labels)

In [123]:
train_x = train.drop(columns = ['type', 'type_num'], axis = 1)
train_y = train['type_num']

test_x = test

In [124]:
train_x.drop(columns = 'fiberID', inplace = True)
test_x.drop(columns = 'fiberID', inplace = True)

In [129]:
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe_train = pd.DataFrame(ohe.fit_transform(train_x[['fiberID2']]).toarray(),
                         columns = ['fiber_0', 'fiber_1', 'fiber_2', 'fiber_3', 'fiber_4', 'fiber_5', 'fiber_6', 'fiber_7'])
ohe_test = pd.DataFrame(ohe.fit_transform(test_x[['fiberID2']]).toarray(),
                        columns = ['fiber_0', 'fiber_1', 'fiber_2', 'fiber_3', 'fiber_4', 'fiber_5', 'fiber_6', 'fiber_7'])
ohe_train.head()

,fiber_0,fiber_1,fiber_2,fiber_3,fiber_4,fiber_5,fiber_6,fiber_7
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [130]:
train_x = pd.concat((train_x, ohe_train), axis = 1)
test_x = pd.concat((test_x, ohe_test), axis = 1)
train_x.drop('fiberID2', axis = 1, inplace = True)
test_x.drop('fiberID2', axis = 1, inplace = True)
train_x.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,...,z_std,distance,fiber_0,fiber_1,fiber_2,fiber_3,fiber_4,fiber_5,fiber_6,fiber_7
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,...,0.478938,11.282572,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,...,0.270199,9.708775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,...,0.194764,9.770866,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,...,0.158362,10.418463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,26.454969,23.058767,21.471406,19.504961,18.389096,256.875910,23.629122,21.742750,19.861718,18.810375,...,0.215129,0.000007,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
#train_x.drop(columns = ['psf_cluster', 'fiber_cluster', 'model_cluster'], inplace = True)
#test_x.drop(columns = ['psf_cluster', 'fiber_cluster', 'model_cluster'], inplace = True)

# Modeling

In [131]:
space = {'objective' : 'multiclass',
         'num_class' : 19,
              'metric' : 'multi_logloss',
              'boosting' : 'gbdt',
    'max_depth':  hp.choice('max_depth', list(range(6, 18, 2))),
    'reg_alpha' : hp.uniform('reg_alpha', 0.01, 0.4),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 0.4),
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.3, 0.9),
    'gamma' : hp.uniform('gamma', 0.01, .7),
    'num_leaves' : hp.choice('num_leaves', list(range(20,150,10))),
    'min_child_samples': hp.choice('min_child_samples', list(range(100,150,10))),
    'subsample': hp.choice('subsample', [.2, .4, .5, .6, .7, .8, .9]),
    'feature_fraction' : hp.uniform('feature_fraction', .4, .8),
    'bagging_fraction' : hp.uniform('bagging_fraction', .4, .9)
}

In [132]:
def evaluate_metric(params):
    time1 = time.time()
    print('Params : {}'.format(params))
    FOLDS = 5
    count = 1
    
    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = 42)
    y_preds = np.zeros(test.shape[0])
    score_mean = 0
    
    for tr_idx, val_idx in skf.split(train_x, train_y):
        X_tr, X_vl = train_x.iloc[tr_idx, :], train_x.iloc[val_idx, :]
        y_tr, y_vl = train_y.iloc[tr_idx], train_y.iloc[val_idx]
        
        lgbtrain = lgb.Dataset(X_tr, label = y_tr)
        lgbval = lgb.Dataset(X_vl, label = y_vl)
        
        lgb_clf = lgb.train(params, lgbtrain, 500, valid_sets = [lgbval],
                           categorical_feature = 'auto',
                           verbose_eval = 0,
                           early_stopping_rounds = 50)
        
        lgb_pred = lgb_clf.predict(X_vl, num_iteration = lgb_clf.best_iteration)
        
        score = make_scorer(log_loss)(lgb_clf, X_vl, y_vl)
        
        score_mean += score
        print('{} CV - Score: {}'.format(count, round(score,4)))
        
        count += 1
    
    time2 = time.time() - time1
    print('Total Time Run : {}'.format(round(time2/60, 2)))
    gc.collect()
    print('Mean Log Loss : {}'.format(score_mean/FOLDS))
    del X_tr, X_vl, y_tr, y_vl, lgb_clf, score
    
    return (score_mean/FOLDS)

In [ ]:
best = fmin(fn = evaluate_metric,
            space = space,
            algo = tpe.suggest,
            max_evals = 20)

best_params = space_eval(space,best)
print("Best HyperParameters :", best_params)

Params : {'bagging_fraction': 0.4068269543439834, 'boosting': 'gbdt', 'colsample_bytree': 0.3787486249141213, 'feature_fraction': 0.5569524460336036, 'gamma': 0.6444364470628207, 'learning_rate': 0.19609979610929865, 'max_depth': 12, 'metric': 'multi_logloss', 'min_child_samples': 100, 'num_class': 19, 'num_leaves': 70, 'objective': 'multiclass', 'reg_alpha': 0.16652438436011124, 'reg_lambda': 0.2314442522984657, 'subsample': 0.5}
1 CV - Score: 0.4046                                                                                                   
2 CV - Score: 0.3974                                                                                                   
3 CV - Score: 0.3929                                                                                                   
4 CV - Score: 0.395                                                                                                    
5 CV - Score: 0.3892                                                                 

1 CV - Score: 0.3664                                                                                                   
2 CV - Score: 0.3687                                                                                                   
3 CV - Score: 0.3672                                                                                                   
4 CV - Score: 0.3718                                                                                                   
5 CV - Score: 0.3708                                                                                                   
Total Time Run : 11.38                                                                                                 
Mean Log Loss : 0.36897419787335045                                                                                    
Params : {'bagging_fraction': 0.7874119384163345, 'boosting': 'gbdt', 'colsample_bytree': 0.3033347474167146, 'feature_fraction': 0.723155451159674, 'gamma': 0.

In [15]:
X_tr, X_vl, y_tr, y_vl = train_test_split(train_x, train_y, test_size = 0.25, random_state = 42, shuffle = True)

X_test = test_x

lgb_train = lgb.Dataset(X_tr, label = y_tr)
lgb_valid = lgb.Dataset(X_vl, label = y_vl)

lgb_model = lgb.train(best_params,
                      lgb_train, 500,
                      valid_sets = [lgb_train, lgb_valid],
                      verbose_eval = 10,
                      categorical_feature = 'auto',
                      early_stopping_rounds = 50)

Training until validation scores don't improve for 50 rounds
[10]	training's multi_logloss: 0.585255	valid_1's multi_logloss: 0.632191
[20]	training's multi_logloss: 0.386771	valid_1's multi_logloss: 0.456104
[30]	training's multi_logloss: 0.322389	valid_1's multi_logloss: 0.413594
[40]	training's multi_logloss: 0.288552	valid_1's multi_logloss: 0.400123
[50]	training's multi_logloss: 0.265227	valid_1's multi_logloss: 0.395356
[60]	training's multi_logloss: 0.246983	valid_1's multi_logloss: 0.393768
[70]	training's multi_logloss: 0.23061	valid_1's multi_logloss: 0.392764
[80]	training's multi_logloss: 0.216745	valid_1's multi_logloss: 0.39266
[90]	training's multi_logloss: 0.203648	valid_1's multi_logloss: 0.393137
[100]	training's multi_logloss: 0.191844	valid_1's multi_logloss: 0.393684
[110]	training's multi_logloss: 0.180877	valid_1's multi_logloss: 0.394391
[120]	training's multi_logloss: 0.170777	valid_1's multi_logloss: 0.395696
Early stopping, best iteration is:
[79]	training's

In [77]:
pred = lgb_model.predict(test_x, num_iteration = lgb_model.best_iteration)

In [91]:
submission = pd.DataFrame(data = pred, columns = sample_submission.columns, index = sample_submission.index)

In [113]:
qso_idx = test_x.loc[(test_x['fiberID2'] == '7') | (test_x['fiberID2'] == '8') | (test_x['fiberID2'] == '9')]

submission.iloc[qso_idx.index] = 0
submission['QSO'].iloc[qso_idx.index] = 1

In [51]:
submission.to_csv('../submission/abc1927.csv')